In [61]:
import pandas as pd
import numpy as np
import os
import json
# import dt
import re

# GM_RESULT

In [64]:
res = pd.read_csv("game_results.csv")
res.head()


,game_id,seed_game_id,players,generations,game_options,scores,createtime
0,g30ccb3351280,NaN,3,6,"{""boardName"":""elysium"",""undoOption"":true,""show...","[{""corporation"":""Splice (breakthrough)"",""corpo...",5/8/2021 20:49:01
1,gbbca1b6090cc,NaN,2,10,"{""boardName"":""elysium"",""undoOption"":true,""show...","[{""corporation"":""Point Luna"",""corporation2"":""V...",5/8/2021 20:54:12
2,gb6144982ca6f,NaN,2,13,"{""boardName"":""hellas"",""undoOption"":true,""showT...","[{""corporation"":""Inventrix"",""corporation2"":"""",...",5/8/2021 21:46:32
3,ga76c9f0c854c,NaN,4,7,"{""boardName"":""tharsis"",""undoOption"":true,""show...","[{""corporation"":""Pobolog (breakthrough)"",""corp...",5/8/2021 22:21:28
4,g832eace87710,NaN,3,8,"{""boardName"":""elysium"",""undoOption"":true,""show...","[{""corporation"":""Valley Trust"",""corporation2"":...",5/8/2021 23:05:02


In [65]:
# preprocessing
res["scores"] = res["scores"].astype(object)
res["game_options"] = res["game_options"].astype(object)
res["createtime"] = pd.to_datetime(res["createtime"], dayfirst=True)


In [66]:
# scores proprocessing
store = []
for i in res["scores"]:
    tmp = json.loads(i)
    for i in range(len(tmp)):
        tmp[i]['order'] = i+1
    tmp = sorted(tmp,key = lambda e:e.__getitem__('playerScore'),reverse = True) # can optimize to obtain knowledge of order
    for i in range(len(tmp)):
        tmp[i]['position'] = i+1
    store.append(tmp)
df1 = pd.DataFrame(store)
df1.columns = ["player1","player2","player3","player4","player5","player6"]

In [67]:
options = []
for i in res["game_options"]:
    if not pd.isna(i):
        options.append(json.loads(i))
    else: options.append({})
df2 = pd.DataFrame(options)

In [68]:
df2.head()

,boardName,undoOption,showTimers,fastModeOption,showOtherPlayersVP,corporateEra,venusNextExtension,coloniesExtension,preludeExtension,turmoilExtension,...,randomMA,soloTR,customCorporationsList,cardsBlackList,customColoniesList,heatFor,breakthrough,doubleCorp,requiresVenusTrackCompletion,requiresMoonTrackCompletion
0,elysium,True,True,False,True,True,True,True,True,False,...,Limited synergy,False,[],[],"[Ceres, Enceladus, Europa, Ganymede, Io, Luna,...",False,True,True,False,False
1,elysium,True,True,False,True,True,True,True,True,False,...,Limited synergy,False,[],[],"[Ceres, Triton, Pluto, Miranda, Venus]",True,True,True,False,False
2,hellas,True,True,True,True,True,False,True,True,False,...,No randomization,False,[],[],"[Ceres, Enceladus, Europa, Ganymede, Io, Luna,...",False,False,False,False,False
3,tharsis,True,True,True,True,True,True,True,True,True,...,Limited synergy,False,[],[],"[Ceres, Enceladus, Europa, Ganymede, Io, Luna,...",True,True,True,False,False
4,elysium,True,True,True,True,True,True,True,True,True,...,No randomization,False,[],[],"[Ceres, Enceladus, Europa, Ganymede, Io, Luna,...",False,False,False,False,False


In [69]:
# df3 = pd.concat([res, df1, df2],axis=1)
# df4= df3.drop(columns=['seed_game_id','game_options','scores','draftVariant','corporateEra','solarPhaseOption','undoOption','fastModeOption','includeVenusMA','soloTR','removeNegativeGlobalEventsOption','politicalAgendasExtension','requiresMoonTrackCompletion','showTimers','politicalAgendasExtensionToggle','aresHazards','requiresVenusTrackCompletion','politicalAgendasExtensionToggle','solarPhaseOption'])

KeyError: "['politicalAgendasExtensionToggle' 'politicalAgendasExtensionToggle'] not found in axis"

In [71]:
df3 = pd.concat([res, df1, df2],axis=1)
df4= df3.drop(columns=['seed_game_id','game_options','scores','draftVariant','corporateEra','solarPhaseOption','undoOption','fastModeOption','includeVenusMA','soloTR','removeNegativeGlobalEventsOption','politicalAgendasExtension','requiresMoonTrackCompletion','showTimers'])

In [72]:
df4.columns
df4.to_csv('./prprocessing_data', index = False)

In [73]:
def getPlayer(df,num): # get all players name
    player_df = pd.DataFrame(columns=['player','count'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                player_df.loc[player_df.shape[0]] = [i[column]['player'].lower().strip(),1]
    ff1 = player_df.groupby(player_df["player"]).count()
    ff1["player_list"] = ff1.values.tolist()
    return ff1


In [74]:
player_count = getPlayer(df4,4)
player_count.to_csv('./pl_count.csv', index = True)

In [75]:
def playerCount(df,num,player): #TODO,模糊查找
    player_df = pd.DataFrame(columns=['id','corporation1', 'rank', 'order','score', 'time', 'generations','corporation2'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                if i[column]['player'].lower() in [p.lower() for p in player] and ('corporation2' not in i[column].keys()):
                    player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],'nan']
                elif i[column]['player'].lower() in [p.lower() for p in player] and ('corporation2' in i[column].keys()):
                    player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],i[column]['corporation2']]
    return player_df

In [76]:
df_player = playerCount(df4,4,['ee'])
df_player.tail(20)

,id,corporation1,rank,order,score,time,generations,corporation2
0,ee,Point Luna,2,1,54,2021-08-15 11:46:55,4,Brotherhood of Mutants


In [77]:
#time series
def time_series(df):
    ff1 = df[(df["players"]==4) & (df["generations"]<=9)]
    ff1.to_csv('./time_original', index = True)
    ff2 = ff1.groupby(ff1['createtime'].dt.date).mean()
    return ff2


In [78]:
# # 时间绘图
# df5 = time_series(df4)
# df5.head(20)
# df5 = df5.drop(columns=["players","initialDraftRounds"])
# df5.to_csv('./time_series.csv', index = True)
# # df5.plot()

KeyError: "['initialDraftRounds'] not found in axis"

# US_RESULT

In [ ]:
# usage: demostrate user's creating time, like "you are the NO.1132 player"
user = pd.read_csv("users.csv")
user = user.drop(columns="prop")
user["name"] = user["name"].str.lower()
user["createtime"] = pd.to_datetime(user["createtime"], dayfirst=True)
user = user.sort_values(by=["createtime"])
user = user.reset_index(drop=True)
user["createtime"] = user.index+1
user.to_csv('./user_pre.csv', index = False)

In [ ]:
# left join and mapping
user_player = user.merge(player_count,how="left",left_on="name",right_on="player")
user_player = user_player.sort_values(by=["player_list"],na_position="first")
user_player.to_csv('./user_player_mapping.csv', index = False)
user_player = user_player.set_index("name")
user_player = user_player.drop(columns="player_list")
user_player.tail()

,createtime,count
name,,
奶牛,83,163.0
阿尔,294,174.0
无语,21,180.0
钝钝,59,286.0
乙木,17,312.0


In [ ]:
# detect missing user name
player_user = player_count.merge(user,how="left",left_on="player",right_on="name")
player_user = player_user.sort_values(by=["count"],ascending=False, na_position="first")
player_user = player_user[player_user["name"].isna()]
player_user.to_csv('./player_user_mapping.csv', index = False)
player_user.head()

,count,player_list,name,createtime
1499,246,[246],NaN,NaN
1354,216,[216],NaN,NaN
1500,123,[123],NaN,NaN
229,120,[120],NaN,NaN
915,112,[112],NaN,NaN


# CUSTOMIZATION

In [ ]:
player_mapping_df = pd.read_csv('player_mapping.csv')
player_mapping_df = player_mapping_df.set_index("playername")

In [ ]:
def playerGroup(df,num,player,map_df): #TODO,模糊查找
    player_df = pd.DataFrame(columns=['id','corporation1', 'rank', 'order','score', 'time', 'generations','corporation2'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                if (i[column]['player'].lower() == player.lower() or i[column]['player'].lower() in map_df.index) and ('corporation2' not in i[column].keys()):
                    if map_df.loc[i[column]['player'].lower(),'username']== player.lower():
                        player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],'nan']
                elif (i[column]['player'].lower() == player.lower() or i[column]['player'].lower() in map_df.index) and ('corporation2' in i[column].keys()):
                    if map_df.loc[i[column]['player'].lower(),'username']== player.lower():
                        player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],i[column]['corporation2']]
    return player_df

In [ ]:
user_df = playerGroup(df4,4,'ender',player_mapping_df)
user_df.head()

,id,corporation1,rank,order,score,time,generations,corporation2
0,ender,Poseidon,4,1,69,2020-08-31 15:05:21,8,nan
1,ender,Philares,1,3,71,2020-08-31 15:05:21,7,nan
2,e神,Thorgate (breakthrough),2,3,82,2020-08-31 15:05:21,8,nan
3,egod,Septem Tribus,1,2,81,2020-08-31 15:05:21,8,nan
4,e神,Valley Trust (breakthrough),2,3,53,2020-08-31 15:05:21,6,nan


In [ ]:
group_by_corp = user_df.groupby(['corporation1']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)

user_df["hour"] = user_df.time.apply(lambda x: x.hour)
group_by_hour = user_df.groupby(['hour']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
group_by_hour.reset_index().loc[0,'hour']
highest = user_df.sort_values(by=["score","generations"],ascending=[False,True])
highest.head()


,id,corporation1,rank,order,score,time,generations,corporation2,hour
105,ender,Point Luna,1,2,151,2021-02-21 18:40:42,9,nan,18
125,ender,Aristarchus,1,2,135,2021-04-13 02:34:31,9,nan,2
77,e神,Thorgate (breakthrough),1,3,124,2020-12-05 23:59:29,9,nan,23
19,ender,Vitor,1,1,116,2020-09-17 17:48:35,9,nan,17
118,ender,Teractor (breakthrough),1,1,113,2021-04-07 01:52:51,8,nan,1


In [ ]:
user_df.to_csv('ee.csv',index=False)

In [ ]:
# test
firstgame = user_df.loc[0,["corporation1","rank","time"]]
user_df["date"] = user_df["time"].dt.normalize()
group_by_date = user_df.groupby(['date']).agg({'id':'count'}).sort_values(by=['id'], ascending=False)
group_by_date = group_by_date[group_by_date["id"]<=7]
group_by_date.reset_index().head()

,date,id
0,2020-10-23,2
1,2021-04-08,2
2,2021-04-24,2
3,2020-12-12,2
4,2020-12-11,2


In [ ]:
def user_summary(ff1,ff2,user): ## ff1=df4
    user_df = playerGroup(ff1,4,user,player_mapping_df)
    # measure 0: check whether the account is uesable
    if user_df.empty: return [''for i in range(18)]
    if pd.isna(ff2.loc[user,"createtime"].astype("int")): return [''for i in range(18)]
    # measure 1: account create date
    createtime = ff2.loc[user,"createtime"]
    # measure 1.5: first game
    firstgame = user_df.loc[0,["corporation1","rank","time"]]
    # measure 1.6 spend how many days, defeat ?% users
    # measure 2: total games, favourite corp
    group_by_corp = user_df.groupby(['corporation1']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
    group_by_corp = group_by_corp.reset_index()
    favorite_corp = group_by_corp.loc[0,'corporation1']
    favourite_corp_rank = group_by_corp.loc[0,'rank']
    favorite_corp_times = group_by_corp.loc[0,'id']
    games = user_df.shape[0]
    # measure 3: most frequently play time period
    user_df["hour"] = user_df.time.apply(lambda x: x.hour)
    group_by_hour = user_df.groupby(['hour']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
    most_freq_time = group_by_hour.reset_index().loc[0,'hour']
    # measure 4: avg score and gen, i.e. defeat 96% players (display this if the ratio > 50) TODO
    avg_rank = user_df["rank"].mean()
    avg_gens = user_df["generations"].mean()
    # measure 5: highest score, with which corp
    # highest = pd.to_numeric(user_df["score"]).idxmax()
    highest = user_df.sort_values(by=["score","generations"],ascending=[False,True])
    highest_corp = highest.iloc[0,1]
    highest_score = highest.iloc[0,4]
    highest_gens = highest.iloc[0,6]
    highest_corp = highest_corp+' and '+user_df.iloc[0,7] if user_df.iloc[0,7] != 'nan' else highest_corp
    # measure 6: group by day. i.e 2021-06-10 seems to be a special day for u, u played 6 games by a row
    user_df["date"] = user_df["time"].dt.normalize()
    group_by_date = user_df.groupby(['date']).agg({'id':'count'}).sort_values(by=['id'], ascending=False)
    group_by_date = group_by_date[group_by_date["id"]<=7]
    played_day = group_by_date.shape[0]
    group_by_date = group_by_date.reset_index()
    most_day = group_by_date.loc[0,["date","id"]]
    # print([user, firstgame[0],firstgame[1],firstgame[2],createtime, games,favorite_corp,favorite_corp_times,most_freq_time,avg_rank,avg_gens, highest_corp, highest_score,highest_gens,played_day,most_day[0], most_day[1]])
    return [user, firstgame[0],firstgame[1],firstgame[2],createtime, games,favorite_corp,favorite_corp_times,favourite_corp_rank,most_freq_time,avg_rank,avg_gens, highest_corp, highest_score,highest_gens,played_day,most_day[0], most_day[1]]

In [ ]:
def measure_summary(ff1,ff2):
    output_df = pd.DataFrame(columns=["user_id","first_corp","first_rank","first_time","create_no","total_games","favorite_corp","fa_corp_times","fa_corp_rank","most_freq_play_time","avg_rank","avg_gens","highest_corp","highest_score","highest_gens","played_day","most_date","most_date_times"])
    for index,i in user_player.iterrows():
        # print(index)
        if (not pd.isnull(i["count"])) and i["count"]>1:
            output_df.loc[output_df.shape[0]] = user_summary(ff1,ff2,index)
    return output_df

In [ ]:
print(user_summary(df4,user_player,'钝钝'))

['钝钝', 'Saturn Systems', 4, Timestamp('2020-08-31 15:05:21'), 59.0, 324, 'Point Luna', 22, 2.1818181818181817, 23, 2.0246913580246915, 7.570987654320987, 'Trade Navigator', 208, 8, 194, Timestamp('2021-04-25 00:00:00'), 6]


In [ ]:
output_df = measure_summary(df4,user_player)
output_df = output_df.dropna()
output_df.to_csv("measure_summary.csv")
output_df.head()


KeyboardInterrupt: 

In [ ]:
def bayesian_avarage(rank,times):
    return (rank*times+2.5*10)/times+10

In [ ]:
output_df = pd.read_csv("measure_summary.csv")
# preprocessing
output_df = output_df[output_df["user_id"].str.contains("tour-")==False]
# play times
output_df = output_df.sort_values(by="total_games",ascending=False)
output_df['play_times_sort'] = range(1,len(output_df)+1)
# gens
output_df = output_df.sort_values(by="avg_gens",ascending=False)
output_df['gens_sort'] = range(1,len(output_df)+1)
# terraform
output_df = output_df.sort_values(by="avg_gens")
output_df['terra_sort'] = range(1,len(output_df)+1)
# terraform
output_df = output_df.sort_values(by="highest_score",ascending=False)
output_df['highest_sort'] = range(1,len(output_df)+1)
# rank
output_df["bayes_rank"] = output_df.apply(lambda x:(x["total_games"]*x["avg_rank"]+1*2.8)/(x["total_games"]+10),axis=1)
output_df = output_df.sort_values(by="bayes_rank")
output_df['rank_sort'] = range(1,len(output_df)+1)
# analyze corp ranking TODO: group by corp,ignore 'breakthrough', then find the index with the maximum values
output_df['baye_avg'] = output_df.apply(lambda x:(x["fa_corp_times"]*x["fa_corp_rank"]+5*2.5)/(x["fa_corp_times"]+5),axis=1)
output_df['max_corp'] = output_df.groupby(["favorite_corp"],as_index=False)["baye_avg"].transform(min)
output_df['max_corp'] = np.where((output_df["max_corp"]==output_df["baye_avg"]) & (output_df["baye_avg"] <= 2.5),1,0)
# output_pivot = pd.pivot_table(output_df, index=[''])
# output_df.sort_values(by="baye_avg").groupby(["favorite_corp"],as_index=False).transform(lambda x: x in range(1,len(output_df)+1))


output_df = output_df.sort_values(by="highest_sort")
output_df.to_csv("measure_summary_sort.csv")
output_df.head()

,Unnamed: 0,user_id,first_corp,first_rank,first_time,create_no,total_games,favorite_corp,fa_corp_times,fa_corp_rank,...,most_date,most_date_times,play_times_sort,gens_sort,terra_sort,highest_sort,bayes_rank,rank_sort,baye_avg,max_corp
292,292,乙木,Manutech,3.0,2020-08-31 15:05:21,17.0,389.0,Point Luna,32.0,2.437500,...,2021-04-11 00:00:00,5.0,2,127,83,1,2.202506,180,2.445946,0
279,279,匆匆,Factorum (breakthrough),4.0,2020-11-07 13:45:03,131.0,78.0,Point Luna,8.0,1.750000,...,2020-11-08 00:00:00,3.0,21,119,91,2,2.259091,186,2.038462,1
265,265,风风,Pobolog (breakthrough),4.0,2021-01-12 23:22:03,109.0,44.0,Vitor,7.0,2.285714,...,2021-04-07 00:00:00,3.0,34,144,67,3,1.681481,138,2.375000,0
244,244,kino,Point Luna,4.0,2020-08-31 15:05:21,1.0,25.0,Point Luna,4.0,2.000000,...,2020-08-31 00:00:00,1.0,58,68,142,4,1.422857,121,2.277778,0
291,291,钝钝,Saturn Systems,4.0,2020-08-31 15:05:21,59.0,307.0,Point Luna,21.0,2.095238,...,2021-04-25 00:00:00,6.0,3,175,35,5,1.974132,164,2.173077,0


In [ ]:
# final json file
final_df = pd.read_csv("measure_summary_sort.csv")
# entity modification
final_df[["create_no","played_day","total_games","fa_corp_times","most_freq_play_time","highest_score","highest_gens","most_date_times"]] = final_df[["create_no","played_day","total_games","fa_corp_times","most_freq_play_time","highest_score","highest_gens","most_date_times"]].astype(np.int32)
final_df["avg_rank"] = final_df["avg_rank"].round(1)
final_df["avg_gens"] = final_df["avg_gens"].round(1)
final_df[["create_time_text","first_game_text","total_game_text","favorite_corp_text","favorite_time_text","avg_rank_text","avg_gen_text","best_score_text","games_in_one_day_text","achievement_text"]] = final_df.apply(lambda x:('','','','','','','','','','',),axis=1,result_type='expand')
# for loop
for i,row in final_df.iterrows():
    # 1, user_id, TODO make it to be index, create_no, create_time_text
    if final_df.loc[i,"create_no"] <= 100: final_df.loc[i,"create_time_text"] = "作为登陆火星的先行者，你们为火星的建设奠定了坚实的基础！"
    elif final_df.loc[i,"create_no"] <= 805: final_df.loc[i,"create_time_text"] = "作为火星的第二批居民，你们牵引了无数小行星让火星不再孤寂寒冷。"
    else: final_df.loc[i,"create_time_text"] = "作为火星的新兴势力，感谢你们为火星增添了几分绿意！"

    # 2, first_time, first_game_text
    date_str = final_df.loc[i,"first_time"]
    final_df.loc[i,"first_time"] = date_str[0:4]+'年'+date_str[5:7]+'月'+date_str[8:10]+'日'
    if final_df.loc[i,"first_rank"].astype(np.int32) == 1: final_df.loc[i,"first_game_text"] = "崭露头角，轻取桂冠"
    elif final_df.loc[i,"first_rank"].astype(np.int32) == 2: final_df.loc[i,"first_game_text"] = "小试牛刀，施精布略"
    elif final_df.loc[i,"first_rank"].astype(np.int32) == 3: final_df.loc[i,"first_game_text"] = "初出茅庐，志在千里"
    elif final_df.loc[i,"first_rank"].astype(np.int32) == 4: final_df.loc[i,"first_game_text"] = "愈战愈勇，砥砺前行"

    # 3, played_day, total_games, total_game_text
    if final_df.loc[i,"total_games"].astype(np.int32) <= 10: final_df.loc[i,"total_game_text"] = "火星好寂寞，下次记得多陪陪人家~"
    elif final_df.loc[i,"total_games"].astype(np.int32) <= 60: final_df.loc[i,"total_game_text"] = "虽然玩得不多，但是改造委员会看到了你的诚意！"
    elif final_df.loc[i,"total_games"].astype(np.int32) <= 120: final_df.loc[i,"total_game_text"] = "改造委员会宣布你是火星年度劳模！"
    else: final_df.loc[i,"total_game_text"] = "地球办公室向你颁发感动火星十大人物证书！"

    # 4, favorite_corp, fa_corp_times
    if final_df.loc[i,"fa_corp_times"].astype(np.int32) >= 10: final_df.loc[i,"favorite_corp_text"] = "日久生情，想必彼此之间已经有羁绊了吧！"
    else: final_df.loc[i,"favorite_corp_text"] = "不要忘记和他一起度过的愉快生活哦~"

    # 5, most_freq_play_time, favorite_time_text
    if final_df.loc[i,"most_freq_play_time"].astype(np.int32) <= 14 and final_df.loc[i,"most_freq_play_time"].astype(np.int32) >= 7: final_df.loc[i,"favorite_time_text"] = "早起的火星人有牌抽！"
    elif final_df.loc[i,"most_freq_play_time"].astype(np.int32) <= 21 and final_df.loc[i,"most_freq_play_time"].astype(np.int32) > 14: final_df.loc[i,"favorite_time_text"] = "真是元气满满的火星人呢"
    else: final_df.loc[i,"favorite_time_text"] = "什么？还有不要睡觉的火星人？你是来自木星的吗？"

    # 6, avg_rank, avg_rank_text
    if final_df.loc[i,"avg_rank"] <= 2: final_df.loc[i,"avg_rank_text"] = "你是出色的技术流！世界政府需要你这样的人才！"
    elif final_df.loc[i,"avg_rank"] <= 3: final_df.loc[i,"avg_rank_text"] = "运气总是起起落落落落落落，我懂你！"
    elif final_df.loc[i,"avg_rank"] <= 4: final_df.loc[i,"avg_rank_text"] = "别灰心！4个人里面只有1个人垫底！"

    # 7, avg_gens, avg_gen_text
    if final_df.loc[i,"avg_gens"] <= 7.2: final_df.loc[i,"avg_gen_text"] = "你是纯纯粹粹的改造党，火卫二不在的第N天，想她"
    elif final_df.loc[i,"avg_gens"] <= 8: final_df.loc[i,"avg_gen_text"] = "你改造拖拖两开花，进可过牌变魔术，退可标动打工人"
    else: final_df.loc[i,"avg_gen_text"] = "你是如假包换的老拖拖，请问AI中心研究的反重力科技成果怎么样了？"

    # 8, highest_corp,highest_score,highest_gens
    if final_df.loc[i,"highest_sort"].astype(np.int32) <= 50: final_df.loc[i,"best_score_text"] = "火星吉尼斯纪录应该留下你的名字！"
    elif final_df.loc[i,"highest_sort"].astype(np.int32) <= 100: final_df.loc[i,"best_score_text"] = "金星总督向你发来祝贺。"
    else: final_df.loc[i,"best_score_text"] = "再接再厉！没事多去研究前哨所转转！"

    # 9 most_date,most_date_times
    date_str = final_df.loc[i,"first_time"]
    final_df.loc[i,"most_date"] = date_str[0:4]+'年'+date_str[5:7]+'月'+date_str[8:10]+'日'
    if final_df.loc[i,"most_date_times"].astype(np.int32) <= 1: final_df.loc[i,"games_in_one_day_text"] = "哦，原来最多也只是开了一局啊，那没事了。"
    elif final_df.loc[i,"most_date_times"].astype(np.int32) <= 3: final_df.loc[i,"games_in_one_day_text"] = "时间管理大师非你莫属！"
    else: final_df.loc[i,"games_in_one_day_text"] = "火星996说的就是你吧！"

    # achievements: play_times_sort,gens_sort,terra_sort,highest_sort,bayes_rank,rank_sort,baye_avg,max_corp
    mapping = ['零','一','二','三','四','五','六','七','八','九','十']
    operated_str = ''
    if final_df.loc[i,"play_times_sort"] <= 10: operated_str += "国服第"+mapping[final_df.loc[i,"play_times_sort"]]+"肝帝\n"

    final_df.loc[i,"achievement"] = operated_str
final_df.to_csv("measure_output.csv")
# final_df.head()